In [210]:
import trimesh

scene = trimesh.Scene()

In [254]:
import my_code.diffusion_training_sign_corr.data_loading as data_loading
import yaml
from tqdm import tqdm



single_dataset, pair_dataset = data_loading.get_val_dataset(
    'SCAPE_a_pair', 'test', 128, preload=False, return_evecs=True, centering='bbox'
)

In [ ]:
# read /lustre/mlnvme/data/s94zalek_hpc-shape_matching/ddpm_checkpoints/single_64_1-2ev_64-128-128_remeshed_fixed/eval/epoch_99/SHREC19_r_pair-test/no_smoothing/2024-11-03_21-44-05/pairwise_results.yaml

with open('/lustre/mlnvme/data/s94zalek_hpc-shape_matching/ddpm_checkpoints/single_64_1-2ev_64-128-128_remeshed_fixed/eval/epoch_99/SHREC19_r_pair-test/no_smoothing/2024-11-03_21-44-05/pairwise_results.yaml', 'r') as f:
    p2p_saved = yaml.load(f, Loader=yaml.FullLoader)
    
p2p_saved

In [255]:
import torch

indx = 1
data_i = pair_dataset[indx]
p2p_i = p2p_saved[indx]
p2p_pairzo = torch.tensor(p2p_i['p2p_median_pairzo'])

print(p2p_i['geo_err_median_pairzo'])

In [237]:
mesh_1 = trimesh.Trimesh(data_i['first']['verts'], data_i['first']['faces'])
mesh_2 = trimesh.Trimesh(data_i['second']['verts'], data_i['second']['faces'])

print(mesh_1.area_faces.sum(), mesh_2.area_faces.sum())

In [261]:
# read /home/s94zalek_hpc/baselines/Spatially-and-Spectrally-Consistent-Deep-Functional-Maps/data/results/SCAPE_a/p2p_21/0_1.txt
import numpy as np

p2p_pairzo = np.loadtxt('/home/s94zalek_hpc/baselines/Spatially-and-Spectrally-Consistent-Deep-Functional-Maps/data/results/SCAPE_a/p2p_21/0_1.txt')
p2p_pairzo = torch.tensor(p2p_pairzo, dtype=torch.int32)

In [260]:
p2p_pairzo

In [ ]:
import my_code.utils.plotting_utils as plotting_utils

scene.geometry.clear()

plotting_utils.plot_p2p_map(
    scene,
    
    
    data_i['first']['verts'], data_i['first']['faces'],
    data_i['second']['verts'], data_i['second']['faces'],
    p2p_pairzo,
    axes_color_gradient=[0, 1],
    base_cmap='hsv'
)

scene.show()

In [264]:
import itertools

list(itertools.combinations(range(3), 2))

In [239]:
import metrics.geodist_metric as geodist_metric
from utils.shape_util import compute_geodesic_distmat

dist_x = torch.tensor(
    compute_geodesic_distmat(data_i['first']['verts'].numpy(), data_i['first']['faces'].numpy())    
)

In [240]:
corr_first = data_i['first']['corr']
corr_second = data_i['second']['corr']

geo_err = geodist_metric.calculate_geodesic_error(
    dist_x, corr_first.cpu(), corr_second.cpu(), p2p_pairzo, return_mean=True
)
geo_err * 100

In [241]:
dist_x.shape

In [242]:
p2p_pairzo.shape

In [243]:
print(corr_first.shape, corr_second.shape)
print(corr_first, corr_second)

In [244]:
data_i['first']['verts'].shape, data_i['second']['verts'].shape

In [245]:
dist_x[p2p_pairzo[corr_second], corr_first].mean()

In [252]:
# read /home/s94zalek_hpc/baselines/Spatially-and-Spectrally-Consistent-Deep-Functional-Maps/data/results/Phi/Phi_tr_reg_080.mat


import scipy.io as sio

mat = sio.loadmat('/home/s94zalek_hpc/baselines/Spatially-and-Spectrally-Consistent-Deep-Functional-Maps/data/results/C/C_tr_reg_080_tr_reg_081.mat')
mat

In [253]:
mat['C'].shape